# ResNet-18 Architecture Graph Implementation in TensorFlow

In [1]:
import numpy as np
import tensorflow as tf

//anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
//anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Building the Graph

In [2]:
print(tf.__version__)

1.8.0


In [3]:
tf.reset_default_graph()

### Inputs, Placeholders, and Constants

In [4]:
NUM_CLASSES = 2
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
x = tf.placeholder(tf.float32, shape=[None,IMAGE_WIDTH,IMAGE_HEIGHT,3]) # represents input 227 x 227 image with 3 color channels (RGB)
y_true = tf.placeholder(tf.float32, shape=[None, NUM_CLASSES])
hold_prob = tf.placeholder(tf.float32)
training = tf.placeholder(tf.bool) # Used for batch normalization - a boolean to indicate whether or not we are training

### Helper Functions

In [5]:
def init_weights(shape):  # initializes the weights randomly with a normal distribution
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape): # inditializes the bias term as a constant of 0.1 values
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)


def max_pool_nbyn(x, name, filter_size=2, stride=2, pad=True):   # creates a max pooling layer
    if pad:
        return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='SAME', name=name)
    else:
        return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='VALID', name=name)
    
def average_pool_nbyn(x, name, filter_size=2, stride=2, pad=True):   # creates a max pooling layer
    if pad:
        return tf.nn.avg_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='SAME', name=name)
    else:
        return tf.nn.avg_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='VALID', name=name)

def global_average_pool(x):
    
    return tf.reshape(tf.reduce_mean(x, [1,2]), [-1, 1, 1, x.get_shape().as_list()[-1]])


def normal_full_layer(input_layer, size):   # creates the fully connected layer
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b  # simple forward propagation using matrix multiplication

def batch_normalization(input_layer, training):  # function for batch normalization
    
    return tf.layers.batch_normalization(input_layer, training=training)

def local_response_normalization(input_layer, radius, alpha, beta, name, bias=1.0): # function for local response normalization
    
     return tf.nn.local_response_normalization(x, depth_radius=radius,
                                              alpha=alpha, beta=beta,
                                              bias=bias, name=name)

def conv(x, filter_height, filter_width, num_filters, stride_y, stride_x, name, pad=None, add_relu=True):
    
    kernel_shape = [filter_height, filter_width]
    #init_random_dist = tf.truncated_normal(kernel_shape, stddev=0.1)
    if pad is not None:
        padded_x = tf.pad(x, [[0, 0], [pad, pad], [pad, pad], [0, 0]])
        conv_layer = tf.layers.conv2d(padded_x, filters=num_filters, 
                            kernel_size=(filter_width, filter_height),
                            strides=(stride_x, stride_y),
                            kernel_initializer=tf.initializers.truncated_normal(stddev=0.1),
                            use_bias=True, bias_initializer=tf.constant_initializer(0.1), name=name)
        if add_relu:
            return tf.nn.relu(conv_layer)
        else:
            return conv_layer
            
    else:
        conv_layer = tf.layers.conv2d(x, 
                            filters=num_filters, 
                            kernel_size=(filter_width, filter_height),
                            strides=(stride_x, stride_y),
                            kernel_initializer=tf.initializers.truncated_normal(stddev=0.1), 
                            use_bias=True, bias_initializer=tf.constant_initializer(0.1), padding="same", name=name)
        
        if add_relu:
            return tf.nn.relu(conv_layer)
        else:
            return conv_layer

### Function to Generate Simple Two-Layer Residual Units 
Based on the following diagram from the ResNet paper.
<img src="resnet_block.png" style="width: 400px; height: 350px;"/>

In [6]:
def residual_unit(input_layer, output_channels, name, pad_zero_dim_match=True):
    
    input_channels = input_layer.get_shape().as_list()[-1] # Gets the number of channels in the input
    
    if input_channels * 2 == output_channels:
        increase_dim = True
        stride = 2
    elif input_channels == output_channels:
        increase_dim = False
        stride = 1
    else:
        raise ValueError('Output and input channel do not match in residual blocks!')
    
    if increase_dim is True:
        if pad_zero_dim_match: # If the we are using the zero-padding approach to match the dimensions
            pooled_input = average_pool_nbyn(input_layer, filter_size=2, stride=2, pad=True, name=(name+'_pooled_input'))
            padded_input = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_channels // 2,
                                                                     input_channels // 2]])
            conv_1 = conv(input_layer, 3, 3, output_channels, 2, 2, name=(name+'_conv_1'), add_relu=False)
            batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
            conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, add_relu=False, name=(name+'_conv_2'))
            
            return tf.nn.relu(batch_normalization(conv_2, training) + padded_input)
        else: # Otherwise the approach for dimension-matching involves 1 x 1 convolutions
            conv_1x1 = conv(input_layer, 1, 1, output_channels, 2, 2,name=(name+'_conv_1x1'), add_relu=false)
            conv_1x1_batch_norm = tf.nn.relu(batch_normalization(conv_1x1))
            
            conv_1 = conv(input_layer, 3, 3, output_channels, 2, 2, name=(name+'_conv_1'), add_relu=False)
            batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
            conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, add_relu=False, name=(name+'_conv_2'))
            
            return tf.nn.relu(batch_normalization(conv_2, training) + conv_1x1_batch_norm)
        
    else:
        conv_1 = conv(input_layer, 3, 3, output_channels, 1, 1, name=(name+'_conv_1'))
        batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
        conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, add_relu=False, name=(name+'_conv_2'))
            
        return tf.nn.relu(batch_normalization(conv_2, training) + input_layer)
        

### CONV 1: 64 7 x 7 filters with stride = 2

In [7]:
conv_1 = tf.nn.relu(batch_normalization(conv(x, 7, 7, num_filters=64, stride_x=2, stride_y=2, name='conv_1', add_relu=False), training))
conv_1.shape

TensorShape([Dimension(None), Dimension(112), Dimension(112), Dimension(64)])

### POOL 1: 3 x 3 filter, stride = 2

In [8]:
pool_1 = max_pool_nbyn(conv_1, filter_size=3, stride=2, pad=True, name='pool_1')
pool_1.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 1: Two stacked 64-filter 3 x 3 conv layers

In [9]:
res_unit_1 = residual_unit(pool_1, 64, name='res_unit_1')
res_unit_1.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 2: Two stacked 64-filter 3 x 3 conv layers

In [10]:
res_unit_2 = residual_unit(res_unit_1, 64, name='res_unit_2')
res_unit_2.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 3: Two stacked 64-filter 3 x 3 conv layers

In [11]:
res_unit_3 = residual_unit(res_unit_2, 64, name='res_unit_3')
res_unit_3.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 4: Two stacked 128-filter 3 x 3 conv layers

In [12]:
res_unit_4 = residual_unit(res_unit_3, 128, name='res_unit_4')
res_unit_4.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 5: Two stacked 128-filter 3 x 3 conv layers

In [13]:
res_unit_5 = residual_unit(res_unit_4, 128, name='res_unit_5')
res_unit_5.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 6: Two stacked 128-filter 3 x 3 conv layers

In [14]:
res_unit_6 = residual_unit(res_unit_5, 128, name='res_unit_6')
res_unit_6.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 7: Two stacked 128-filter 3 x 3 conv layers

In [15]:
res_unit_7 = residual_unit(res_unit_6, 128, name='res_unit_7')
res_unit_7.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 8: Two stacked 256-filter 3 x 3 conv layers

In [16]:
res_unit_8 = residual_unit(res_unit_7, 256, name='res_unit_8')
res_unit_8.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### AVG POOL: Global Average Pooling

In [17]:
avg_pool = global_average_pool(res_unit_8)
avg_pool.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(256)])

### FC : Fully connected layer 

In [18]:
avg_pool_flattened = tf.reshape(avg_pool, [-1, 256])
fc = normal_full_layer(avg_pool_flattened, NUM_CLASSES)
fc.shape

TensorShape([Dimension(None), Dimension(2)])

### Final Output

In [19]:
y_pred = fc
y_pred.shape

TensorShape([Dimension(None), Dimension(2)])

### Loss Function with Softmax Activation

In [20]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Adam Optimizer

In [21]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001) # Adam optimizer
train = optimizer.minimize(cross_entropy)  # training operation

init = tf.global_variables_initializer() # global variables initializer